# Imports 

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import TextVectorization, Embedding, SimpleRNN, Dense, LSTM, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
import multiprocessing
from mlflow import MlflowClient
import mlflow
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
from Source.preprocess_data import *  ## import all functions from preprocess_data.py
from Source.postprocess_data import * ## import all functions from postprocess_data.py
from Source.utils import *  ## import all functions from utils.py
# import nltk
# import optuna
# 🤗
from transformers import TFAutoModel, AutoTokenizer



from sklearn.metrics import confusion_matrix 
# from nltk.corpus import stopwords  
from tqdm import tqdm
tqdm.pandas()
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

nw = multiprocessing.cpu_count()



client = MlflowClient(tracking_uri="http://localhost:8080")
os.environ["TF_KERAS"]='1'
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("GPUs disponibles :", tf.config.list_physical_devices("GPU"))
print("Version TF :", tf.__version__)

c:\Users\bassm\.conda\envs\AI_env_P7_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.10.1
Num GPUs Available:  1
GPUs disponibles : [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Version TF : 2.10.1


# Préparation data

## Importation dataset

In [2]:
df = pd.read_csv('https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/AI+Engineer/Project+7%C2%A0-+D%C3%A9tectez+les+Bad+Buzz+gr%C3%A2ce+au+Deep+Learning/sentiment140.zip',
                header=None,
                compression='zip',
                encoding='cp1252')

df.columns = ['target', 'ids', 'date', 'flag', 'user', 'text']

sample_df, _ = train_test_split(df, test_size=0.9, random_state=42, stratify=df['target'])
sample_df = sample_df.reset_index(drop=True)
print(f"Sample size: {sample_df.shape[0]} rows")
# On ne garde que les colonnes 'target' et 'text'
sample_df = sample_df[['target', 'text']]
sample_df["target"] = sample_df["target"].apply(lambda x: 0 if x == 0 else 1)
sample_df.to_csv('Data/raw_data.csv', index=False)

Sample size: 160000 rows


## Train/Validation split

In [3]:
# Data
X_raw = sample_df['text']
y = sample_df['target']
X_train, X_val, y_train, y_val = train_test_split(X_raw, y, test_size=0.2, random_state=42, stratify=y)


# Fonction centrale 

In [4]:
rnn_size = 64
epochs = 10
lr = 1e-3


def test_bert_model(bert_model_name):
    with mlflow.start_run():
        mlflow.log_params(params={
            'rnn_size': rnn_size, 
            'epochs': epochs, 
            'learning_rate': lr,
            'bert_model_name':bert_model_name
        })

        tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
        encodings_train = tokenizer(X_train.to_list(), truncation=True, padding=True, return_tensors="tf")
        encodings_val = tokenizer(X_val.to_list(), truncation=True, padding=True, return_tensors="tf")

        dataset_train = tf.data.Dataset.from_tensor_slices(
            (
                {"input_ids": encodings_train["input_ids"], 
                 "attention_mask": encodings_train["attention_mask"]
                 },y_train
                )
                ).batch(32)
        
        dataset_val = tf.data.Dataset.from_tensor_slices(
            (
                {"input_ids": encodings_val["input_ids"], 
                 "attention_mask": encodings_val["attention_mask"]
                 },y_val
                )
                ).batch(32)
        # On charge le modèle pré-entrainné
        base_model = TFAutoModel.from_pretrained(bert_model_name, from_pt=True)
        base_model.trainable = False # Pas de fine-tuning ou d'entrainement car impossible à faire avec les ressources disponibles

        # Construction du modèle keras 
        ## Une input layer pour les input ids
        input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name="input_ids")
        ## Une input layer pour le masque d'attention
        attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name="attention_mask")
        ## On récupère 

        outputs = base_model(input_ids, attention_mask=attention_mask)
        # On prend le token [CLS] comme vecteur de phrase
        if "roberta" in model_name or "bertweet" in model_name:
        # Roberta / BERTweet utilisent également last_hidden_state[:,0,:] pour le [CLS]
            cls_token = outputs.last_hidden_state[:, 0, :]
        else:
            cls_token = outputs.last_hidden_state[:, 0, :]
    
        x = tf.keras.layers.Dense(rnn_size, activation="relu")(cls_token)
        x = tf.keras.layers.Dropout(0.2)(x)
        logits = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    
        model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=logits)

        ## Callbacks
        model_savepath = f"./Models/MY_{'_'.join(bert_model_name.split('/'))}_dense{rnn_size}.h5"
        checkpoint = ModelCheckpoint(model_savepath, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min')
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10)
        lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=0, min_lr=1e-5)
        callbacks_list = [checkpoint, es, lr_scheduler]


        optimizer = Adam(learning_rate=lr)
        model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
        # Summary
        model.summary()
        # History
        with tf.device("/GPU:0"):
            history = model.fit(dataset_train, epochs=epochs, batch_size=64, validation_data=dataset_val, callbacks=callbacks_list, verbose=1)


        model.load_weights(model_savepath)

                # Prédictions sur le jeu de validation
        y_pred_proba = model.predict(dataset_val)
        y_pred = (y_pred_proba>0.5)


        output_dict = postprocess_model_output(y_val, y_pred, y_pred_proba) # voir postprocess_data.py

        # Logging des métriques dans MLflow
        mlflow.log_metrics(output_dict)
        # Matrice de confusion
        cm = confusion_matrix(y_val, y_pred, normalize='pred')
        fig, ax = plt.subplots()
        sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues", ax=ax, )
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.title("Confusion Matrix - Validation Set")
        fig.savefig("confusion_matrix.png")
        plt.close(fig)
        mlflow.log_artifact("confusion_matrix.png")
        #
        fig2 = plot_training_history(history,show=False)
        fig2.savefig("learning_path.png")
        plt.close(fig2)
        mlflow.log_artifact("learning_path.png")

        # Enregistrement du modèle dans MLflow
        mlflow.tensorflow.log_model(model, "model")
    




# Experiment MLFlow

In [5]:
# Création de l'étude Optuna et optimisation
print("Setting up MLflow experiment...")
mlflow.set_experiment("BERT_models_experiment")
exp_id = mlflow.get_experiment_by_name("BERT_models_experiment").experiment_id

experiment_description = (
    "Comparaison de plusieurs modèles BERT"
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "Sentiment analysis modelling",
    "model_type": "BERT_pretrained",
    "team": "Ph. Constant",
    "project_quarter": "Q3-2025",
    "mlflow.note.content": experiment_description,
}

for key, value in experiment_tags.items():
    client.set_experiment_tag(exp_id, key, value)

Traceback (most recent call last):
  File "c:\Users\bassm\.conda\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 366, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "c:\Users\bassm\.conda\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 464, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "c:\Users\bassm\.conda\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1634, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "c:\Users\bassm\.conda\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1627, in _read_helper
    result = read_yaml(root, file_name)
  File "c:\Users\bassm\.conda\envs\AI_env_P7_gpu\lib\site-packages\mlflow\utils\yaml_utils.py", line 107, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' does not exist."

Setting up MLflow experiment...


In [ ]:
bert_model_list = [
    # "prajjwal1/bert-tiny",
    # "prajjwal1/bert-small",
    "distilbert-base-uncased",
    "roberta-base",
    "distilroberta-base",
    "vinai/bertweet-base",  
    "finiteautomata/bertweet-base-sentiment-analysis"
]

# Lancement experiment

In [7]:
for model_name in bert_model_list:
    print(f"Running test with BERT model : {model_name}")
    test_bert_model(model_name)

4000/4000 [==============================] - 4386s 1s/step - loss: 0.5505 - accuracy: 0.7213 - val_loss: 0.4971 - val_accuracy: 0.7562 - lr: 0.0010
Epoch 2/10
4000/4000 [==============================] - 4593s 1s/step - loss: 0.5150 - accuracy: 0.7435 - val_loss: 0.4857 - val_accuracy: 0.7661 - lr: 0.0010
Epoch 3/10
4000/4000 [==============================] - 4541s 1s/step - loss: 0.5093 - accuracy: 0.7491 - val_loss: 0.4585 - val_accuracy: 0.7880 - lr: 0.0010
Epoch 4/10
4000/4000 [==============================] - 4549s 1s/step - loss: 0.5061 - accuracy: 0.7511 - val_loss: 0.4555 - val_accuracy: 0.7881 - lr: 0.0010
Epoch 5/10
4000/4000 [==============================] - 4552s 1s/step - loss: 0.5034 - accuracy: 0.7533 - val_loss: 0.4602 - val_accuracy: 0.7878 - lr: 0.0010
Epoch 6/10
4000/4000 [==============================] - 4332s 1s/step - loss: 0.5035 - accuracy: 0.7526 - val_loss: 0.4839 - val_accuracy: 0.7576 - lr: 0.0010
Epoch 7/10
4000/4000 [==============================] - 4

2025/09/17 01:24:00 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/17 01:24:00 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmp5sph3p35\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmp5sph3p35\model\data\model\assets
2025/09/17 01:25:28 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\bassm\AppData\Local\Temp\tmp5sph3p35\model, flavor: tensorflow). Fall back to return ['tensorflow==2.10.1']. Set logging level to DEBUG to see the full traceback. 
2025/09/17 01:25:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Running test with BERT model : distilroberta-base


...

c:\Users\bassm\.conda\envs\AI_env_P7_gpu\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\bassm\.conda\envs\AI_env_P7_gpu\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\bassm\.cache\huggingface\hub\models--finiteautomata--bertweet-base-sentiment-analysis. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Deve

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, None)]       0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 tf_roberta_model_3 (TFRobertaM  TFBaseModelOutputWi  134899968  ['input_ids[0][0]',              
 odel)                          thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, None                                         

2025/09/17 18:02:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/17 18:02:02 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmpcu7ge4yc\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmpcu7ge4yc\model\data\model\assets
2025/09/17 18:03:56 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\bassm\AppData\Local\Temp\tmpcu7ge4yc\model, flavor: tensorflow). Fall back to return ['tensorflow==2.10.1']. Set logging level to DEBUG to see the full traceback. 
2025/09/17 18:03:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


# Optimisation de la tête du modèle (plus petit dataset) 

## Réduction du dataset d'entrainement

In [8]:
sample_df, _ = train_test_split(df, test_size=0.99, random_state=42, stratify=df['target'])
sample_df = sample_df.reset_index(drop=True)
print(f"Sample size: {sample_df.shape[0]} rows")
# On ne garde que les colonnes 'target' et 'text'
sample_df = sample_df[['target', 'text']]
sample_df["target"] = sample_df["target"].apply(lambda x: 0 if x == 0 else 1)
sample_df.to_csv('Data/raw_data_mini.csv', index=False)

# Data
X_raw = sample_df['text']
y = sample_df['target']
X_train, X_val, y_train, y_val = train_test_split(X_raw, y, test_size=0.2, random_state=42, stratify=y)


Sample size: 16000 rows


Fonction de base

In [9]:
epochs = 50
lr = 1e-3


def test_bert_model_v2(bert_model_name, rnn_size):
    with mlflow.start_run():
        mlflow.log_params(params={
            'rnn_size': rnn_size, 
            'epochs': epochs, 
            'learning_rate': lr,
            'bert_model_name':bert_model_name
        })

        tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
        encodings_train = tokenizer(X_train.to_list(), truncation=True, padding=True, return_tensors="tf")
        encodings_val = tokenizer(X_val.to_list(), truncation=True, padding=True, return_tensors="tf")

        dataset_train = tf.data.Dataset.from_tensor_slices(
            (
                {"input_ids": encodings_train["input_ids"], 
                 "attention_mask": encodings_train["attention_mask"]
                 },y_train
                )
                ).batch(32)
        
        dataset_val = tf.data.Dataset.from_tensor_slices(
            (
                {"input_ids": encodings_val["input_ids"], 
                 "attention_mask": encodings_val["attention_mask"]
                 },y_val
                )
                ).batch(32)
        # On charge le modèle pré-entrainné
        base_model = TFAutoModel.from_pretrained(bert_model_name, from_pt=True)
        base_model.trainable = False # Pas de fine-tuning ou d'entrainement car impossible à faire avec les ressources disponibles

        # Construction du modèle keras 
        ## Une input layer pour les input ids
        input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name="input_ids")
        ## Une input layer pour le masque d'attention
        attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name="attention_mask")
        ## On récupère 

        outputs = base_model(input_ids, attention_mask=attention_mask)
        # On prend le token [CLS] comme vecteur de phrase
        if "roberta" in model_name or "bertweet" in model_name:
        # Roberta / BERTweet utilisent également last_hidden_state[:,0,:] pour le [CLS]
            cls_token = outputs.last_hidden_state[:, 0, :]
        else:
            cls_token = outputs.last_hidden_state[:, 0, :]
    
        x = tf.keras.layers.Dense(rnn_size, activation="relu")(cls_token)
        x = tf.keras.layers.Dropout(0.2)(x)
        logits = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    
        model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=logits)

        ## Callbacks
        model_savepath = f"./Models/MY_{'_'.join(bert_model_name.split('/'))}_dense{rnn_size}.h5"
        checkpoint = ModelCheckpoint(model_savepath, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min')
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10)
        lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=0, min_lr=1e-5)
        callbacks_list = [checkpoint, es, lr_scheduler]


        optimizer = Adam(learning_rate=lr)
        model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
        # Summary
        model.summary()
        # History
        with tf.device("/GPU:0"):
            history = model.fit(dataset_train, epochs=epochs, batch_size=64, validation_data=dataset_val, callbacks=callbacks_list, verbose=1)


        model.load_weights(model_savepath)

                # Prédictions sur le jeu de validation
        y_pred_proba = model.predict(dataset_val)
        y_pred = (y_pred_proba>0.5)


        output_dict = postprocess_model_output(y_val, y_pred, y_pred_proba) # voir postprocess_data.py

        # Logging des métriques dans MLflow
        mlflow.log_metrics(output_dict)
        # Matrice de confusion
        cm = confusion_matrix(y_val, y_pred, normalize='pred')
        fig, ax = plt.subplots()
        sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues", ax=ax, )
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.title("Confusion Matrix - Validation Set")
        fig.savefig("confusion_matrix.png")
        plt.close(fig)
        mlflow.log_artifact("confusion_matrix.png")
        #
        fig2 = plot_training_history(history,show=False)
        fig2.savefig("learning_path.png")
        plt.close(fig2)
        mlflow.log_artifact("learning_path.png")

        # Enregistrement du modèle dans MLflow
        mlflow.tensorflow.log_model(model, "model")
    




In [12]:
# Création de l'étude Optuna et optimisation
print("Setting up MLflow experiment...")
mlflow.set_experiment("BERT_models_experiment")
exp_id = mlflow.get_experiment_by_name("BERT_models_experiment").experiment_id

experiment_description = (
    "Optimisation de la tête du modèle basé sur BERT."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "Sentiment analysis modelling",
    "model_type": "BERT_pretrained",
    "team": "Ph. Constant",
    "project_quarter": "Q3-2025",
    "mlflow.note.content": experiment_description,
}

for key, value in experiment_tags.items():
    client.set_experiment_tag(exp_id, key, value)

Setting up MLflow experiment...


Traceback (most recent call last):
  File "c:\Users\bassm\.conda\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 366, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "c:\Users\bassm\.conda\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 464, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "c:\Users\bassm\.conda\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1634, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "c:\Users\bassm\.conda\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1627, in _read_helper
    result = read_yaml(root, file_name)
  File "c:\Users\bassm\.conda\envs\AI_env_P7_gpu\lib\site-packages\mlflow\utils\yaml_utils.py", line 107, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' does not exist."

In [ ]:
for rnn_size in list(range(64,254,64)):
    print(f"Running test with BERT model finiteautomata/bertweet-base-sentiment-analysis")
    test_bert_model_v2(bert_model_name="finiteautomata/bertweet-base-sentiment-analysis", rnn_size=rnn_size)

400/400 [==============================] - 149s 373ms/step - loss: 0.3990 - accuracy: 0.8241 - val_loss: 0.3833 - val_accuracy: 0.8316 - lr: 0.0010
Epoch 5/50
400/400 [==============================] - 160s 400ms/step - loss: 0.3953 - accuracy: 0.8236 - val_loss: 0.3844 - val_accuracy: 0.8328 - lr: 0.0010
Epoch 6/50
400/400 [==============================] - 171s 428ms/step - loss: 0.3933 - accuracy: 0.8272 - val_loss: 0.3778 - val_accuracy: 0.8322 - lr: 0.0010
Epoch 7/50
400/400 [==============================] - 151s 377ms/step - loss: 0.3894 - accuracy: 0.8284 - val_loss: 0.3820 - val_accuracy: 0.8359 - lr: 0.0010
Epoch 8/50
400/400 [==============================] - 152s 380ms/step - loss: 0.3872 - accuracy: 0.8291 - val_loss: 0.3814 - val_accuracy: 0.8328 - lr: 0.0010
Epoch 9/50
400/400 [==============================] - 166s 416ms/step - loss: 0.3829 - accuracy: 0.8285 - val_loss: 0.3745 - val_accuracy: 0.8366 - lr: 5.0000e-04
Epoch 10/50
400/400 [==============================] 

2025/09/18 00:35:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/18 00:35:23 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmpyl_jlx48\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmpyl_jlx48\model\data\model\assets
2025/09/18 00:36:51 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\bassm\AppData\Local\Temp\tmpyl_jlx48\model, flavor: tensorflow). Fall back to return ['tensorflow==2.10.1']. Set logging level to DEBUG to see the full traceback. 
2025/09/18 00:36:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Running test with BERT model finiteautomata/bertweet-base-sentiment-analysis


...

c:\Users\bassm\.conda\envs\AI_env_P7_gpu\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['roberta.embeddings.position_ids', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, None)]       0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 tf_roberta_model_11 (TFRoberta  TFBaseModelOutputWi  134899968  ['input_ids[0][0]',              
 Model)                         thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, None                                        

2025/09/18 03:04:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/18 03:04:12 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmpq936dckt\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmpq936dckt\model\data\model\assets
2025/09/18 03:05:24 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\bassm\AppData\Local\Temp\tmpq936dckt\model, flavor: tensorflow). Fall back to return ['tensorflow==2.10.1']. Set logging level to DEBUG to see the full traceback. 
2025/09/18 03:05:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


# Modèle Bert automatique depuis Huggingface 


In [38]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# Charger le tokenizer et le modèle automatiquement (transformers choisit RobertaTokenizer / TFRobertaForSequenceClassification)
tokenizer = AutoTokenizer.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")
model = TFAutoModelForSequenceClassification.from_pretrained(
                                                        "finiteautomata/bertweet-base-sentiment-analysis", 
                                                        num_labels=2,
                                                        ignore_mismatched_sizes=True )

for layer in model.layers:
    if layer.name != "classifier":   # nom de la couche finale
        layer.trainable = False

# Vérification

model.summary()


# Tokenisation
encodings_train = tokenizer(X_train.to_list(), truncation=True, padding=True, return_tensors="tf")
encodings_val = tokenizer(X_val.to_list(), truncation=True, padding=True, return_tensors="tf")

dataset_train = tf.data.Dataset.from_tensor_slices(
    (
        {
        "input_ids": encodings_train["input_ids"], 
        "attention_mask": encodings_train["attention_mask"]
        },y_train
    )
    ).batch(32)
        
dataset_val = tf.data.Dataset.from_tensor_slices(
    (
        {
        "input_ids": encodings_val["input_ids"], 
        "attention_mask": encodings_val["attention_mask"]
        },y_val
    )
    ).batch(32)


optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)  # LR plus élevé car seules quelques couches sont entraînées
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

metrics = ["accuracy"]

rename = '_'.join("finiteautomata/bertweet-base-sentiment-analysis".split('/'))
        ## Callbacks
model_savepath = f"./Models/AUTO_{rename}.h5"
checkpoint = ModelCheckpoint(model_savepath, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=0, min_lr=1e-5)
callbacks_list = [checkpoint, es, lr_scheduler]


model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Exemple d’entraînement
history = model.fit(dataset_train, validation_data=dataset_val, epochs=50,callbacks=callbacks_list, verbose=1)

print(model.config.id2label)


c:\Users\bassm\.conda\envs\AI_env_P7_gpu\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading file vocab.txt from cache at C:\Users\bassm/.cache\huggingface\hub\models--finiteautomata--bertweet-base-sentiment-analysis\snapshots\924fc4c80bccb8003d21fe84dd92c7887717f245\vocab.txt
loading file bpe.codes from cache at C:\Users\bassm/.cache\huggingface\hub\models--finiteautomata--bertweet-base-sentiment-analysis\snapshots\924fc4c80bccb8003d21fe84dd92c7887717f245\bpe.codes
loading file added_tokens.json from cache at C:\Users\bassm/.cache\huggingface\hub\models--finiteautomata--bertweet-base-sentiment-analysis\snapshots\924fc4c80bccb8003d21fe84dd92c7887717f245\added_tokens.json
loading file special_tokens_map.json from cache at C:\Users\bassm/.cache\huggingface\hub\models--finite

Model: "tf_roberta_for_sequence_classification_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLayer  multiple                 134309376 
 )                                                               
                                                                 
 classifier (TFRobertaClassi  multiple                 592130    
 ficationHead)                                                   
                                                                 
Total params: 134,901,506
Trainable params: 592,130
Non-trainable params: 134,309,376
_________________________________________________________________
Epoch 1/50
400/400 [==============================] - 165s 388ms/step - loss: 0.4843 - accuracy: 0.7817 - val_loss: 0.4481 - val_accuracy: 0.8003 - lr: 0.0010
Epoch 2/50
400/400 [==============================] - 140s 350ms/step - loss: 0.4605 - accuracy: 0.7930 - val_l